# AB SEO Test Summary

In [6]:
!pip install pyathena

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
from pyathena import connect

In [8]:
# from smart_open import smart_open

In [9]:


import pandas as pd
import numpy as np
from datetime  import datetime, timedelta, date
import time


from athena_querying import *
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [13]:
from athena_common_queries import *

In [12]:
reload athena_common_queries

SyntaxError: invalid syntax (<ipython-input-12-38c9e7a59c4d>, line 1)

In [14]:
query = create_blog_pageview_and_scroll_query("SG", datetime(2019,12,10), datetime.now())

Getting the pageview and read events from 2019-12-10T00:00:00 to 2019-12-12T11:04:51.616538


In [15]:
print(query)


    SELECT
      CAST("from_iso8601_timestamp"("sent_at") AS timestamp) "sent_at_timestamp"
    , "sent_at"
    , "type"
    , "body"."event_name"
    , "body"."data"."status"
    , (CASE "body"."data"."status" 
        WHEN 'Page Bottom Reached' THEN 100 
        WHEN 'Article Reading Completed' THEN 100 WHEN 'Article Body 100' THEN 100 
        WHEN 'Article Reading 75%' THEN 75 WHEN 'Article Body 75' THEN 75 
        WHEN 'Article Reading 50%' THEN 50 WHEN 'Article Body 50' THEN 50 
        WHEN 'Article Reading 25%' THEN 25 WHEN 'Article Body 25' THEN 25
        WHEN 'Article Reading Started' THEN 0 WHEN 'Article Loaded' THEN 0 
        ELSE 0 END) "article_read_depth"
    , "user"."anonymous_id"
    , "user"."amp_id"
    
 
    , "context"."page_url"
    , "context"."referrer"
    , CAST("strpos"("context"."page_url", '?amp') AS boolean) "is_amp"
    , CAST((("context"."page_url" LIKE '%utm_medium%') OR ("context"."page_url" LIKE '%gclid%')) AS boolean) "has_marketing_param"
    

## Settings

In [7]:
run_speed_tests = False

# Using Athena Querying convenience

In [16]:

aq = AthenaQuery()
aq.connect()


In [ ]:
# Get AB Results Summary for SG

# Get Results

In [8]:
if run_speed_tests:
    athena_conn, athena_cursor = connect_to_athena()
    print("Checking athena works")
    query = "select * from ms_data_stream_production_processed limit 100"
    test_query_results = pd.read_sql(query, athena_conn)

    test_query_results.head(10)

In [9]:
# blog_pageview_metrics
# blog_pageviews_per_user_per_day

# Speed Testing

In [10]:
speed_test_query = "select * from "+athena_database +".blog_pageview_metrics order by day_str, is_test"

In [11]:
# Original / obvious way (may be dependent on imports that have been changed in other files)
if run_speed_tests:
    start_time = datetime.now()
    
    pageview_metrics_results = pd.read_sql(speed_test_query, athena_conn)
    end_time = datetime.now()
    execution_seconds_conventional = (end_time-start_time).total_seconds()
    print("Running the old way took %.2f seconds"%execution_seconds_conventional)
    pageview_metrics_results.head()


In [12]:
# TODO: sounds like you can speed up athena massively by querying from the S3 bucket, 
# rather than getting results from the connection object.  There's some issue with long running queries and large results.
# https://github.com/laughingman7743/PyAthena/issues/46
# https://stackoverflow.com/questions/53262533/why-pyathena-doesnt-work-on-longer-running-queries-while-athena-runs-them/53324034

In [13]:
# Using pandas cursor 
if run_speed_tests:
    ap_start_time = datetime.now()
    athena_cursor.execute(speed_test_query)
    ap_query_done_time = datetime.now()
    query_duration_seconds = (ap_query_done_time - ap_start_time).total_seconds()
    print("Query finished after %.2f" % query_duration_seconds)
    
    df = as_pandas(athena_cursor)
    ap_done_time = datetime.now()
    ap_total_seconds = (ap_done_time - ap_start_time).total_seconds()
    print("All finished using as_pandas in %.2f"%ap_total_seconds)
    df.head()

In [14]:
# Querying the S3 results directly
if run_speed_tests:
    # assume query has been run and timed already
    # Note that this way it doesn't have context on the data types, but I'm not sure glue was set up properly for that anyway.
    s3_start_time = datetime.now()
    bucket, key = athena_cursor.output_location.strip('s3://').split('/', 1)
    print(bucket,key)
    session = boto3.session.Session()

    # result = session.resource('s3').Object(bucket, key)
    s3_client = boto3.client('s3')
    response = s3_client.get_object(Bucket=bucket,Key=key)
    
    file = response["Body"]

    df = pd.read_csv(file)
    s3_end_time = datetime.now()
    s3_read_seconds = (s3_end_time - s3_start_time).total_seconds()
    print("Reading the CSV from S3 (not running query) took %.2f"%s3_read_seconds)

In [15]:
df

NameError: name 'df' is not defined

In [25]:
for c in df.dtypes: print(c)

object
object
object
object
object
int64
bool
int64


In [31]:
df = df.astype({
    "anonymous_id":"string_",
    "canonical_url":"string_",
    "day_str":"string_",
    "from_time":"datetime64",
   "to_time":"datetime64",
    "time_on_page_seconds":"float16",
    "is_test":"bool",
    "max_article_read_depth":"int16",
    
}, copy=False)


In [32]:
for c in df.dtypes: print(c)

object
object
object
datetime64[ns]
datetime64[ns]
float16
bool
int16


In [12]:
pyathena.__version__

'1.8.0'

In [34]:
# Run a test query to see that the function works

df = query_athena_via_s3(speed_test_query, athena_cursor, s3_client, print_debug_messages=True)
print("Num records = %i"% len(df))
df.head(10)

Running query
Query finished after 16.73
Now going to get the results from S3
Reading the CSV from S3 (not running query) took 272002.44
All finished using as_pandas in 18.89
Num records = 389406


,anonymous_id,canonical_url,day_str,from_time,to_time,time_on_page_seconds,is_test,max_article_read_depth
0,85da893c-245e-46db-9892-3e36d4a47fe0,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 22:34:44.142,2019-10-28 22:38:25.310,221,False,100
1,e083f2c1-7459-4bae-98e0-0de99464e816,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 22:56:40.997,2019-10-28 22:57:49.543,68,False,100
2,0581c41d-2310-45cc-9c95-c082e418de02,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 16:38:07.689,2019-10-28 16:39:21.032,73,False,100
3,0629cb02-8f5e-4a3d-bf6d-0820e116cb94,https://www.moneysmart.tw/articles/%E5%AD%98%E...,2019-10-28,2019-10-28 16:26:48.853,2019-10-28 16:27:25.412,36,False,100
4,8cc3e00a-6fe6-4c83-b6fd-bb06d0f328b6,https://www.moneysmart.tw/articles/%e9%89%85%e...,2019-10-28,2019-10-28 16:16:48.076,2019-10-28 16:16:48.076,0,False,0
5,66ee0aaf-15d7-49d8-89e1-80eeecc0dcc9,https://www.moneysmart.tw/articles/%E5%8F%B0%E...,2019-10-28,2019-10-28 19:19:49.531,2019-10-28 19:20:11.895,22,False,100
6,50afd6f9-5cbc-49e9-bc2c-fba42a46cc59,https://www.moneysmart.tw/articles/netflix-lin...,2019-10-28,2019-10-28 19:36:28.890,2019-10-28 19:37:34.584,65,False,100
7,cc386a09-4cdd-4aca-b77e-139599922f24,https://www.moneysmart.tw/articles/%E5%9C%B0%E...,2019-10-28,2019-10-28 16:56:29.927,2019-10-28 16:56:29.951,0,False,100
8,b524ec2f-069b-4c82-a80f-d4008adc14f4,https://www.moneysmart.tw/articles/2019-%E6%84...,2019-10-28,2019-10-28 16:57:01.905,2019-10-28 16:57:01.905,0,False,0
9,3d14032a-6d30-4acd-bb26-944dcbd97a53,https://www.moneysmart.tw/articles/costco%E8%B...,2019-10-28,2019-10-28 22:11:53.677,2019-10-28 22:11:53.677,0,False,0


# Hacky AMP Pageviews Per Session Trend in AB Test


In [ ]:
#athena = AthenaQuery()

# Get the Test URLs


# Generate a humongous query
# Pageviews per user who has visited an SEO test url.

# Get the results


